# Background
...this is your ML take home exercise. You have 90 mins to complete. Don't worry if you don't make it through everything... The exercise is designed to be time constrained. __Please leave your code in the cells and annotate or provide context where possible__. The goal of this exercise is to see how you solve problems, not assess your answers. Have fun!

## Question 1
####Background Info
- In paid search, for a given search word, the ads for multiple companies may be displayed
- In this case, there are ads from three companies being displayed 
- We talk about the order in which these ads are displayed as the "position" of an Ad
- Ads in a lower position are displayed higher on the page and are generally more expensive, but also recieve more clicks
- "Impressions" are the number of instances that an online advertisement is displayed
- Impressions are equivalent between ads in different positions
- A "Click" is an instance of someone clicking on an advertisement
- A "Transaction" is an instance of someone sending money


####Scenario
- You are currently in position 2 of a paid search term, the click-through rate (customer clicks per impression) is 1% and the cost per click is \$0.75. 
- You can get into position 1 by paying \$1.25 per click. In position 1, you expect a click-though rate of 1.5\%. 
- In position 3 you would pay \$0.25 cost per click, and have a click-though rate of 0.5\%
- Conversion rate (transactions per click) once someone clicks is 10% in all 3 positions
- Our total revenue per transfer (before search costs) is \$15
- Our goal is to maximize absolute margin \$ (revenue from transactions - search costs of those transactions)

####Questions
(1.A) What is the cost per first transaction in position 2?

(1.B) Which of the 3 positions do you want to be in?

(1.C) At what click through rate in position 1 are you indifferent between positions 1 and 2?


In [ ]:
%matplotlib inline
#1B
#Our goal is to maximize rev from transaction - search cost. Lets consider 1000 impressions:
num_impressions = 1000
total_rev_per_transaction = 15 
conv_rate = 0.1
#position1
click_rate_p1 = 0.015
cost_click_p1 = 1.25
num_clicks_p1 = num_impressions * click_rate_p1 
conv_thousand_p1 = num_impressions * click_rate_p1 * conv_rate
cost_of_clicks_p1 = num_clicks_p1 * cost_click_p1
rev_p1 = conv_thousand_p1 * total_rev_per_transaction
margin_p1 = rev_p1 - cost_of_clicks_p1

#position2
click_rate_p2 = 0.01
cost_click_p2 = 0.75
num_clicks_p2 = num_impressions * click_rate_p2 
conv_thousand_p2 = num_impressions * click_rate_p2 * conv_rate
cost_of_clicks_p2 = num_clicks_p2 * cost_click_p2
rev_p2 = conv_thousand_p2 * total_rev_per_transaction
margin_p2 = rev_p2 - cost_of_clicks_p2

#position3
click_rate_p3 = 0.005
cost_click_p3 = 0.25
num_clicks_p3 = num_impressions * click_rate_p3 
conv_thousand_p3 = num_impressions * click_rate_p3 * conv_rate
cost_of_clicks_p3 = num_clicks_p3 * cost_click_p3
rev_p3 = conv_thousand_p3 * total_rev_per_transaction
margin_p3 = rev_p3 - cost_of_clicks_p3
margins = [margin_p1, margin_p2, margin_p3]
print("best position is:", margins.index(max(margins)) +  1)



best position is: 2


In [ ]:
#1A
cost_tenclicks_p2 = cost_click_p2/conv_rate
print("cost for ten clicks in position 2 is: ", cost_tenclicks_p2)


cost for ten clicks in position 2 is:  7.5


In [ ]:
#1C (I did not convert to Python but it is simple math)
At what ctr you are indifferent between 1 and 2
Mrgin = Imp * CTR * 15 (rev)* .10 - Imp * CTR * CPC
MArgin = Imp*CTR (1.5 - CPC)
Margin 2 = Margin 1 
.75 = CTR_1(1.5 - CPC_1)
.75 = CTR_1(0.25)
CTR_1 =  3% 


#Question 2

### Hat Sales at Remitly Island Resort

Hat sales drive an enormous amount of revenue at the Remitly Island Resort. While walking around the resort on your first day as an employee, you overhear several customers complaining about some hat stands running out of hats midday. You also notice stands with hats left over at closing time.

Your job is to build a predictive model to tell us how many hats we should stock at each stand on a given day.

Walk us through your process of building a model from beginning to end (EDA, model selection, crossval, etc.). __Make sure to leave as much commentary as you can and your code inline__... Our goal is to learn how you think about problems!


| Column  | Data Type  | Definition  |
|---|---|---|
| **hat_sales** | Float  |  [Target Variable] The number of hats sold per day.|
| **employees_scheduled**  |  Float |  The number of employees working at the resort. |
| **temperature** | Float  | Temperature at the resort.  |
| **total_num_tourists**  | Float  | Number of tourists at the resort.  |
| **total_num_vehicles**  | Float  | Number vehicles at the resort.  |
| **probability_of_rain**  |  Float | Probability of rain at the resort.  |
| **location**  | String  | Location at the resort where hats were sold.  |

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/jakeweholt/remitly_takehome_dataset/master/resort_hat_sales.csv")

In [ ]:
df.head()

,hat_sales,employees_scheduled,temperature,total_num_tourists,total_num_vehicles,probability_of_rain,location
0,41.0,118.0,44.557300,87.0,35.0,0.699233,game_room
1,21.0,103.0,63.762835,46.0,19.0,0.371571,pool
2,137.0,117.0,40.904708,87.0,38.0,0.359049,pool
3,23.0,119.0,53.888821,62.0,26.0,0.802631,game_room
4,22.0,108.0,65.062831,34.0,13.0,0.786027,game_room


So the problem is a classic demand prediction problem, we should build a model to predict demand of hats at each hat stand. 
We first start by doing some EDA:
Let's first take a look at Missing Values

In [ ]:
df.isnull().sum()

hat_sales              0
employees_scheduled    0
temperature            0
total_num_tourists     0
total_num_vehicles     0
probability_of_rain    0
location               0
dtype: int64

Good news!! we don't need to dal with missing values. So next step will be to take a look at sufficient statistics of inputs:


In [ ]:
df.mean()

hat_sales               67.586000
employees_scheduled    109.868000
temperature             50.840598
total_num_tourists      68.701000
total_num_vehicles      49.234000
probability_of_rain      0.581923
dtype: float64

In [ ]:
df.var()

hat_sales               4935.780384
employees_scheduled       32.785361
temperature              160.480078
total_num_tourists       761.865464
total_num_vehicles     16461.378623
probability_of_rain        0.033540
dtype: float64

hmm, it seems thaty probability of rain convey little information (variance is small compared to mean) so we can keep this in mind while training the model

# **Pre-processing**

# New Section

# New Section

Luckily :) we do not have missing values, and the data is fairly simple so we need to just handle categorical values and then normalizations. Generally speaking, I have to do outlier removal and detection may be using mean +/- 3std as well, but for now I am skipping it to respect the 90min limit

In [ ]:
categorical_cols = ['location']

In [ ]:
df_cat = pd.get_dummies(df)

In [ ]:
df_cat.columns

Index(['hat_sales', 'employees_scheduled', 'temperature', 'total_num_tourists',
       'total_num_vehicles', 'probability_of_rain', 'location_game_room',
       'location_lobby', 'location_pool'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
normalized_data = mms.fit_transform(df_cat[df_cat.columns[1:]])
df_norm = pd.DataFrame(normalized_data, columns=df_cat.columns[1:])

In [ ]:
df_norm.mean()

employees_scheduled    0.493400
temperature            0.343099
total_num_tourists     0.731784
total_num_vehicles     0.040229
probability_of_rain    0.576161
location_game_room     0.345000
location_lobby         0.373000
location_pool          0.282000
dtype: float64

In [ ]:
df_norm.var()

employees_scheduled    0.081963
temperature            0.021576
total_num_tourists     0.013679
total_num_vehicles     0.011451
probability_of_rain    0.034471
location_game_room     0.226201
location_lobby         0.234105
location_pool          0.202679
dtype: float64

In the above, I used a feature scaling method in order to make sure all the features have the same
energy and reduce the risk of over-fitting. This can be done by normalizing each column by range of
that column as follwos:

**Training**

I splitted the data into test and train splits, then I'll use linear models to perform regression. I use RidgeCV which has built-in cross validation. An alternative way is to use linear regression or ridge regression and use cross validation separately. 



In [ ]:
X, y = df_norm, df_cat['hat_sales']

In [ ]:
# training, validation split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .1, random_state=12)

In [ ]:
y_val

518    115.0
871     66.0
797     27.0
274     33.0
325     13.0
       ...  
403     88.0
886     40.0
542     16.0
29     236.0
116     42.0
Name: hat_sales, Length: 100, dtype: float64

In [ ]:
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
# I test two different approaches: Linear Rigression (linear model) and SVR with
# GAussian Kernel for non-linear model; Doing a proper model selection on
# validation set, one can select the best perfoming model


clf = linear_model.RidgeCV(cv=5)
clf_nonlinear = SVR(C=50, epsilon=0.1)
#since there are only 8 features, we use them all for now
# for demonstration, I use linear model in pipeline
pipeline = Pipeline([
		('selector', SelectKBest(chi2, k='all')),
		('classifier', clf)
		])

In [ ]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('selector',
                 SelectKBest(k='all',
                             score_func=<function chi2 at 0x7f2c6880ebf8>)),
                ('classifier',
                 SVR(C=50, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     shrinking=True, tol=0.001, verbose=False))],
         verbose=False)

In [ ]:
clf_nonlinear.score(X_train, y_train)
# this score is R^2 coeeficient of regression model

0.34389908517721957

In [ ]:
y_predicted = pipeline.predict(X_val)

In [ ]:
from sklearn.metrics import mean_squared_log_error
mean_squared_error(y_val, y_predicted)
# this is actual MSE of the estimatior, we might want to use mean of log square error instead. 

4192.465469816075

In [ ]:
y_val

518    115.0
871     66.0
797     27.0
274     33.0
325     13.0
       ...  
403     88.0
886     40.0
542     16.0
29     236.0
116     42.0
Name: hat_sales, Length: 100, dtype: float64

In [ ]:
y_predicted

array([ 64.41937544,  72.69907579,  49.060445  ,  48.97068009,
         6.22651384,  66.84620277,  15.99396503,  80.57037897,
        32.5951098 ,  79.45735683,  32.26705132,  87.83306145,
        62.96550595,  27.16697041,  51.2829838 ,  89.96221614,
       115.3586258 ,  38.41366508,  59.80823413, -11.37865975,
        40.66513136,  15.81906247,  71.20178277,  65.00849226,
        67.8410746 ,  45.52291224,  66.64103011,  78.04266259,
        56.76149545,  61.52006712,  68.76541451,  53.50398216,
        72.34884506,  65.85555356,  44.20951129,  57.42812835,
        69.28625595,  22.5535471 ,  59.63933377,  81.80391383,
        60.17616611,  37.07244145, 104.67883261, 102.61516489,
        69.54776968,  95.06738517,  15.90512948,  78.84229134,
        59.5954776 ,  84.01100484,   3.28250377,  42.46895218,
        53.38206539,  17.11751847,   3.95746654,  17.62252534,
        10.22722453,  79.16627587,  76.44661469,  56.748656  ,
        47.26258148,  29.55943762,  22.9155235 , -13.00

You can see some of the values are negative, this might be indicating that the linear assumption is not valid. So, by using non-linear model this can be resolbed. Another way to handle this is to use log of sale as target variable, this is called transformation in modern regression literature. Or simply, we can wrapp the predictor inside a sign function. Given that we have more time, next step is to do parameter tuning (model parameters, number of features), model selection, feature selection method, etc. using a validation set

r
